<a href="https://colab.research.google.com/github/cam2149/AnalisisDatosI/blob/main/Ejercicio_merge_join_US_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
# 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv'
# 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv'
# 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv'

# Debemos encontrar el ranking de los estados de USA  por su densidad de población en  2010

# 1. many-to-one merge nombre + población
# 2. merge on ``state/region``  de  ``population``,y ``abbreviation`` de ``abbrevs``, ``how='outer'`` para tener todos los datos


In [136]:
# Importar las bibliotecas de Python requeridas.
import pandas as pd
import numpy as np

# Visualización
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style = 'whitegrid')
# missing value imputation
from sklearn.impute import SimpleImputer

# pipeline & transformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

Objetivo: "Analizar  los datos relevantes de los dataset entregados.

**Dataset state-population.csv**
Cada fila representa un registro de la poblacion del estado correspondiente según grupación de menores de 18 y el total de la población.

**Dataset state-areas.csv**
Cada fila representa un registro de la área del estado de los Estados Unidos correspondiente.

**Dataset state-abbrevs.csv**
Cada fila representa un registro la abrevitura del nombre de cada estado de los Estados Unidos.

Con la realación de los datos encontraremos
1. many-to-one merge nombre + población
2. merge on ``state/region``  de  ``pop``,y ``abbreviation`` de ``abbrevs``.
 ``how='outer'`` para tener todos los datos


In [137]:
# Cargar los datasets desde sus fuentes.
# Como desconocemos si los archivos contienen caracteres ?, con este comando na_values = '?' pandas los interpreta como nan (not a number).
state_population = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv',na_values = '?')
state_areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv',na_values = '?')
state_abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv',na_values = '?')

## 2. Verificamos los DF

## state_population

In [138]:
state_population.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [139]:
# Obtener la cantidad de registros por columna del DataFrame state_population.
state_population.count()

,0
state/region,2544
ages,2544
year,2544
population,2524


In [140]:
state_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544 entries, 0 to 2543
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state/region  2544 non-null   object 
 1   ages          2544 non-null   object 
 2   year          2544 non-null   int64  
 3   population    2524 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 79.6+ KB


In [141]:
# Verificar si existen datos vacíos o faltantes en el DataFrame stata_population.
state_population.isna().sum()

,0
state/region,0
ages,0
year,0
population,20


In [142]:
# Promedio/media de la columna population
avg_population = state_population["population"].astype("float").mean()

# Imprimir los resultados
print("Promedio de population:", avg_population)


Promedio de population: 6805558.401347068


In [143]:
# Reemplazo de NaN por la media, columna population
state_population["population"].replace(np.nan, avg_population, inplace=True)

In [144]:
# Verificar si existen datos duplicados en el DataFrame state_population.
print(f'Cantidad de datos duplicados en el DataFrame state_population: {state_population.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame state_population: 0


In [145]:
# Verificar si existen datos vacíos o faltantes en el DataFrame stata_population.
state_population.isna().sum()

,0
state/region,0
ages,0
year,0
population,0


## state-areas

In [146]:
# Obtener el cabecero del DataFrame state-areas.
state_areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [147]:
# Obtener la cantidad de registros por columna del DataFrame state_areas.
state_areas.count()

,0
state,52
area (sq. mi),52


In [148]:
# Verificar si existen datos vacíos o faltantes en el DataFrame state_areas.
state_areas.isna().sum()

,0
state,0
area (sq. mi),0


In [149]:
# Verificar si existen datos duplicados en el DataFrame state_areas.
print(f'Cantidad de datos duplicados en el DataFrame state_areas: {state_areas.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame state_areas: 0


## state_abbrevs

In [150]:
# Obtener el cabecero del DataFrame state_abbrevs.
state_abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [151]:
# Obtener la cantidad de registros por columna del DataFrame state_abbrevs.
state_abbrevs.count()

,0
state,51
abbreviation,51


In [152]:
# Verificar si existen datos vacíos o faltantes en el DataFrame state_abbrevs.
state_abbrevs.isna().sum()

,0
state,0
abbreviation,0


In [153]:
# Verificar si existen datos duplicados en el DataFrame state_abbrevs.
print(f'Cantidad de datos duplicados en el DataFrame state_abbrevs: {state_abbrevs.duplicated().sum()}')

Cantidad de datos duplicados en el DataFrame state_abbrevs: 0


## Intregración y/o relacionamiento de los Dataframes

In [154]:
# Renombrar la columna state/region a abbreviation del Datafame state_population
state_population.rename(columns={'state/region': 'abbreviation'}, inplace=True)

In [155]:
# Realizar el merge de los DataFrame state_areas y state_abbrevs mediante la columna state
merged_dataframes = state_areas.merge(state_abbrevs, how = 'outer', on = 'state')
merged_dataframes

,state,area (sq. mi),abbreviation
0,Alabama,52423,AL
1,Alaska,656425,AK
2,Arizona,114006,AZ
3,Arkansas,53182,AR
4,California,163707,CA
5,Colorado,104100,CO
6,Connecticut,5544,CT
7,Delaware,1954,DE
8,Florida,65758,FL
9,Georgia,59441,GA


In [156]:
# Obtener la cantidad de registros por columna del DataFrame resultado.
merged_dataframes.count()

,0
state,52
area (sq. mi),52
abbreviation,51


In [157]:
# Verificar si existen datos vacíos o faltantes en el DataFrame resultado.
merged_dataframes.isna().sum()

,0
state,0
area (sq. mi),0
abbreviation,1


In [158]:
# Verificar si existen datos duplicados en el merged_dataframes resultado.
print(f'Cantidad de datos duplicados en el merged_dataframes resultado: {merged_dataframes.duplicated().sum()}')

Cantidad de datos duplicados en el merged_dataframes resultado: 0


In [159]:
# Identificar el valor isna en dataframe merged_dataframes

merged_dataframes[merged_dataframes.isna().any(axis=1)]


,state,area (sq. mi),abbreviation
51,Puerto Rico,3515,NaN


In [160]:
# Reemplazo de NaN por la media, columna abbreviation
merged_dataframes["abbreviation"].replace(np.nan, 'PR', inplace=True)

In [161]:
merged_dataframes.isna().sum()

,0
state,0
area (sq. mi),0
abbreviation,0


In [162]:
# Obtener la cantidad de registros por columna del DataFrame resultado.
merged_dataframes.count()

,0
state,52
area (sq. mi),52
abbreviation,52


In [163]:
# Realizar el merge de los DataFrame state_areas y state_abbrevs mediante la columna state
merged_dataframes = merged_dataframes.merge(state_population, how = 'right', on = 'abbreviation')
merged_dataframes

,state,area (sq. mi),abbreviation,ages,year,population
0,Alabama,52423.0,AL,under18,2012,1117489.0
1,Alabama,52423.0,AL,total,2012,4817528.0
2,Alabama,52423.0,AL,under18,2010,1130966.0
3,Alabama,52423.0,AL,total,2010,4785570.0
4,Alabama,52423.0,AL,under18,2011,1125763.0
...,...,...,...,...,...,...
2539,NaN,NaN,USA,total,2010,309326295.0
2540,NaN,NaN,USA,under18,2011,73902222.0
2541,NaN,NaN,USA,total,2011,311582564.0
2542,NaN,NaN,USA,under18,2012,73708179.0


In [164]:
merged_dataframes.count()

,0
state,2496
area (sq. mi),2496
abbreviation,2544
ages,2544
year,2544
population,2544


In [165]:
merged_dataframes[merged_dataframes.isna().any(axis=1)]

,state,area (sq. mi),abbreviation,ages,year,population
2496,NaN,NaN,USA,under18,1990,64218512.0
2497,NaN,NaN,USA,total,1990,249622814.0
2498,NaN,NaN,USA,total,1991,252980942.0
2499,NaN,NaN,USA,under18,1991,65313018.0
2500,NaN,NaN,USA,under18,1992,66509177.0
2501,NaN,NaN,USA,total,1992,256514231.0
2502,NaN,NaN,USA,total,1993,259918595.0
2503,NaN,NaN,USA,under18,1993,67594938.0
2504,NaN,NaN,USA,under18,1994,68640936.0
2505,NaN,NaN,USA,total,1994,263125826.0


In [166]:
# prompt: Crea un transformador para el campo area (sq. mi) donde se totalice el area (sq. mi) segun el year y se asigne el valor al campo area (sq. mi) cuyo dato es Nan segun el year

# Crear un transformador para el campo area (sq. mi)
def total_area(df):
  # Calcular la suma del area (sq. mi) por año
  area_by_year = df.groupby('year')['area (sq. mi)'].sum()

  # Asignar el valor total al campo area (sq. mi) donde sea NaN
  for year, area in area_by_year.items():
    df.loc[(df['year'] == year) & (df['area (sq. mi)'].isnull()), 'area (sq. mi)'] = area

  return df

# Aplicar el transformador al DataFrame
merged_dataframes = total_area(merged_dataframes)


In [167]:
# Reemplazo de NaN por la media, columna abbreviation
merged_dataframes["state"].replace(np.nan, 'UnitedState', inplace=True)

In [168]:
merged_dataframes.count()

,0
state,2544
area (sq. mi),2544
abbreviation,2544
ages,2544
year,2544
population,2544


In [169]:
merged_dataframes

,state,area (sq. mi),abbreviation,ages,year,population
0,Alabama,52423.0,AL,under18,2012,1117489.0
1,Alabama,52423.0,AL,total,2012,4817528.0
2,Alabama,52423.0,AL,under18,2010,1130966.0
3,Alabama,52423.0,AL,total,2010,4785570.0
4,Alabama,52423.0,AL,under18,2011,1125763.0
...,...,...,...,...,...,...
2539,UnitedState,7580798.0,USA,total,2010,309326295.0
2540,UnitedState,7580798.0,USA,under18,2011,73902222.0
2541,UnitedState,7580798.0,USA,total,2011,311582564.0
2542,UnitedState,7580798.0,USA,under18,2012,73708179.0


In [134]:
# Histograma de population del merged_dataframes solo con los ages con el valor total y en el eje x aplicar el campo year y el y campo population,
# excluye la abbreviation con los valores USA

import altair as alt
alt.Chart(merged_dataframes[(merged_dataframes.ages == 'total') & (merged_dataframes.abbreviation != 'USA')]).mark_bar().encode(
    alt.X('year:O'),
    alt.Y('sum(population):Q')
)


alt.Chart(...)